# Creating DataFrames with Python

In [2]:
# import pyspark class Row from module sql
from pyspark.sql import *

# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)

# Create the DepartmentWithEmployees instances from Departments and Employees
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

print department1
print employee2
print departmentWithEmployees1.employees[0].email

In [3]:
# Create the first DataFrame from a list of the rows.
departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2]
df1 = sqlContext.createDataFrame(departmentsWithEmployeesSeq1)

display(df1)

In [4]:
# Create a second DataFrame from a list of rows.
departmentsWithEmployeesSeq2 = [departmentWithEmployees3, departmentWithEmployees4]
df2 = sqlContext.createDataFrame(departmentsWithEmployeesSeq2)

display(df2)

# Working with DataFrames

In [6]:
# Union 2 DataFrames.
unionDF = df1.unionAll(df2)
display(unionDF)

In [7]:
# Write the Unioned DataFrame to a Parquet file.
# Remove the file if it exists
dbutils.fs.rm("/tmp/df-example.parquet", True)
unionDF.write.parquet("/tmp/df-example.parquet")

In [8]:
# Read a DataFrame from the Parquet file.
parquetDF = sqlContext.read.parquet("/tmp/df-example.parquet")
display(parquetDF)

In [9]:
# Explode the employees column.
from pyspark.sql import Row
from pyspark.sql import functions as F
eDF = sqlContext.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])

print eDF.select(F.explode(eDF.intlist).alias("anInt")).collect()
eDF.select(F.explode(eDF.mapfield).alias("key", "value")).show()

In [10]:
from pyspark.sql.functions import explode
df = parquetDF.select(explode("employees").alias("e"))
explodeDF = df.selectExpr("e.firstName", "e.lastName", "e.email", "e.salary")
display(explodeDF)

In [11]:
# Use filter() to return only the rows that match the given predicate.
filterDF = explodeDF.filter(explodeDF.firstName == "xiangrui").sort(explodeDF.lastName)
display(filterDF)

In [12]:
from pyspark.sql.functions import col, asc
# use | instead of or
filterDF = explodeDF.filter((col("firstName") == "xiangrui") | (col("firstName") == "michael"))
display(filterDF)

In [13]:
# The where() clause is equivalent to filter().
whereDF = explodeDF.where((col("firstName") == "xiangrui") | (col("firstName") == "michael")).sort(asc("lastName"))
display(whereDF)

In [14]:
# Replace null values with -- using DataFrame Na functions.
nonNullDF = explodeDF.fillna("--")
display(nonNullDF)

In [15]:
# Retrieve only rows with missing firstName or lastName.
filterNullDF = explodeDF.filter((col("firstName").isNull()) | (col("lastName").isNull())).sort("email")
display(filterNullDF)

In [16]:
# Example aggregations using agg() and countDistinct().
from pyspark.sql.functions import countDistinct
countDistinctDF = explodeDF.select("firstName", "lastName").groupBy("firstName", "lastName").agg(countDistinct("firstName"))
display(countDistinctDF)

In [17]:
# Compare the DataFrame and SQL Query Physical Plans (Hint: They should be the same.)
countDistinctDF.explain()

In [18]:
explodeDF.registerTempTable("table_example")
countDistinctDF_sql = sqlContext.sql("SELECT firstName, lastName, count(distinct firstName) as distinct_first_names FROM table_example GROUP BY firstName, lastName")
countDistinctDF_sql.explain()

In [19]:
# Sum up all the salaries
salarySumDF = explodeDF.agg({"salary": "sum"})
display(salarySumDF)

In [20]:
# Print the summary statistics for the salaries.
explodeDF.describe("salary").show()

In [21]:
display(explodeDF.select("salary"))

In [22]:
# An example using Pandas & Matplotlib Integration
import pandas as pd
import matplotlib.pyplot as plt
plt.clf()
pdDF = nonNullDF.toPandas()
pdDF.plot(x="firstName", y="salary", kind="bar", rot=45)
display()

In [23]:
# Cleanup: Remove the parquet file.
dbutils.fs.rm("/tmp/df-example.parquet", True)